In [ ]:
# gettting the range of the intensity and the number of pepetides
import pandas as pd
from pathlib import Path
import numpy as np
direcroy_path = '/media/kusterlab/internal_projects/active/TOPAS/WP31/Playground/Retrospective_study/2023.07.12_CJ_PDX_mouse_human'
transcripts_path = "/media/kusterlab/internal_projects/active/TOPAS/WP31/FPKM/fpkms_221219mitINFORM.csv"
# intensity files
PREPROCESSED_FP_INTENSITY = 'preprocessed_fp.csv'
REGEX = r'(^\S+-.+-\S)|(Reporter intensity corrected)|(^P\d{6}$)'
Z_SCORES_PATH = 'full_proteome_measures_z.tsv'
Z_SCORES_PATH_pp = 'phospho_measures_z.tsv'
Z_SCORE_REGEX = r'(^\S+-.+-\S)|(Reporter intensity corrected)|(^P\d{6}$) Z-score'
BASKET_SCORES_PATH = 'basket_scores_4th_gen.tsv'

REGEX_META = r'^Identification metadata'  # to get num identified pepetides in FP intensity file
def replace_statement(x):
    x = x.replace('num_peptides=','')
    x = x.replace('|','')
    x = x.replace(';','')
    x = x.replace('detected in batch','0')
    return x


def unnest_proteingroups(df:pd.DataFrame) -> pd.DataFrame:
    """
    Unnest the protein_groups A;B as two separate rows with the same values
    the protein groups are the index of the the pandas dataframe df
    """
    temp_df = df
    temp_df['index'] = temp_df.index.str.split(';')
    temp_df = temp_df.explode('index')
    temp_df = temp_df.set_index('index')
    return temp_df


def intersection(lst1, lst2):
    return list(set(lst1) & set(lst2))


def setdiff(lst1,lst2):
    return(list(set(lst1) - set(lst2)))


def get_correlation(protein:str,df1,df2):
    ans = None

    try:
        df1 = pd.DataFrame(df1.loc[protein,:])
        df2 = pd.DataFrame(df2.loc[protein,:])
        ans = df1.corrwith(df2)[0]
    except:
        print(f'{protein} lead to error we put NA')
        pass
    return ans

excel_file = '/home/amir/Downloads/Heatmap_Afatinib resistance.xlsx'
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import colors
#  full proteome_z_scores
Z_SCORES_df_proteome = pd.read_csv(Path(direcroy_path)/Path(Z_SCORES_PATH),sep='\t')
Z_SCORES_df_proteome = Z_SCORES_df_proteome.set_index('Gene names')
Z_SCORES_df_proteome = unnest_proteingroups(Z_SCORES_df_proteome)
Z_SCORES_df_proteome.columns = Z_SCORES_df_proteome.columns.str.replace('zscore_','')
df = pd.read_excel(excel_file)
df = df.set_index('Sample  name')
protein_names = df.columns.tolist()
patients = df.index.tolist()
subset = Z_SCORES_df_proteome[Z_SCORES_df_proteome.index.isin(protein_names)]
patients = [x for x in patients if x in subset.columns]
subset = subset[patients]
subset.to_excel('FP.xlsx')
sns.heatmap(subset)
plt.rcParams["figure.figsize"] = [20, 20]
plt.rcParams["figure.autolayout"] = True
plt.savefig('Fp_level.svg')
plt.show()

Z_SCORES_df_pp = pd.read_csv(Path(direcroy_path)/Path(Z_SCORES_PATH_pp),sep='\t')
Z_SCORES_df_pp = Z_SCORES_df_pp.set_index('Modified sequence')
Z_SCORES_df_pp.columns = Z_SCORES_df_pp.columns.str.replace('zscore_','')
df = pd.read_excel(excel_file,sheet_name='heatmap 2')
df = df.set_index('Sample  name')
protein_names = df.columns.tolist()
patients = df.index.tolist()
subset = Z_SCORES_df_pp[Z_SCORES_df_pp.index.isin(protein_names)]
patients = [x for x in patients if x in subset.columns]
subset = subset[patients]
sns.heatmap(subset)
plt.rcParams["figure.figsize"] = [20, 20]
plt.rcParams["figure.autolayout"] = True
plt.savefig('pp_level.svg')
plt.show()